## Load data

In [1]:
import pandas as pd

In [7]:
football_data_path = r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\data\football_data.csv"
df_footie = pd.read_csv(football_data_path)
df_footie

,team,league,country,url
0,Arsenal,Premier League,England,https://ng.soccerway.com//teams/england/arsena...
1,Manchester City,Premier League,England,https://ng.soccerway.com//teams/england/manche...
2,Newcastle United,Premier League,England,https://ng.soccerway.com//teams/england/newcas...
3,Manchester United,Premier League,England,https://ng.soccerway.com//teams/england/manche...
4,Tottenham Hotspur,Premier League,England,https://ng.soccerway.com//teams/england/totten...
...,...,...,...,...
93,Getafe,La Liga,Spain,https://ng.soccerway.com//teams/spain/getafe-c...
94,Celta Vigo,La Liga,Spain,https://ng.soccerway.com//teams/spain/real-clu...
95,Real Valladolid,La Liga,Spain,https://ng.soccerway.com//teams/spain/real-val...
96,Cadiz,La Liga,Spain,https://ng.soccerway.com//teams/spain/cadiz-cl...


## Scape

In [6]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
# import streamlit as st
# import json

In [69]:
agent = 'Mozilla/5.0 (Windows NT 10.0; Windows; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'
url = df_footie[df_footie['team'] == 'Manchester United'].url.values[0]
response = requests.get(url, headers={'User-Agent': agent})

In [70]:
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find_all('table', class_='matches')
dates = [row.text for row in table[0].find_all('td', class_="full-date")]
leagues = [row.text.strip() for row in table[0].find_all('td', class_="competition")]
homes = [row.text.strip() for row in table[0].find_all('td', class_="team")[::2]]
aways = [row.text.strip() for row in table[0].find_all('td', class_="team")[1::2]]
scores = [row.text.strip() for row in table[0].find_all('td', class_="score-time")]

# Create Empty dataframe
df_fixtures = pd.DataFrame(
    {
        'Date': dates,
        'League': leagues,
        'Home team': homes,
        'Time': scores,
        'Away team': aways
    }
)

df_fixtures_prev5 = df_fixtures[:5]
df_fixtures_next5 = df_fixtures[-5:] 

In [72]:
df_fixtures_prev5

,Date,League,Home team,Time,Away team
0,17/03/24,FAC,Manchester United,E\n 4 - 3\n E,Liverpool
1,30/03/24,PRL,Brentford,1 - 1,Manchester United
2,04/04/24,PRL,Chelsea,4 - 3,Manchester United
3,07/04/24,PRL,Manchester United,2 - 2,Liverpool
4,13/04/24,PRL,AFC Bournemouth,2 - 2,Manchester United


In [71]:
df_fixtures_next5

,Date,League,Home team,Time,Away team
5,21/04/24,FAC,Coventry City,16 : 30,Manchester United
6,24/04/24,PRL,Manchester United,21 : 00,Sheffield United
7,27/04/24,PRL,Manchester United,16 : 00,Burnley
8,06/05/24,PRL,Crystal Palace,21 : 00,Manchester United
9,12/05/24,PRL,Manchester United,17 : 30,Arsenal


In [39]:
from datetime import datetime, timedelta

# Function to subtract two hours from the time string
def subtract_n_hours(time_str, n):

    # Remove empty spaces
    time_str = time_str.replace(' ', '')

    # Convert string to datetime object
    datetime_obj = datetime.strptime(time_str, '%H:%M')
    
    # Subtract n hours
    datetime_obj -= timedelta(hours=n)
    
    # Format the datetime object as a string in '%I:%M %p' format
    return datetime_obj.strftime('%I:%M %p')

In [43]:
df_fixtures_next5['Time'] = df_fixtures_next5['Time'].apply(lambda x: subtract_n_hours(x, 8))

C:\Users\Owner\AppData\Local\Temp\ipykernel_57356\2313276544.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixtures_next5['Time'] = df_fixtures_next5['Time'].apply(lambda x: subtract_n_hours(x, 8))


In [45]:
# Define the function to convert the date string
def convert_date(date_str):
    # Define the input format of the given date string
    input_format = '%d/%m/%y'
    
    # Define the desired output format for the date
    output_format = '%b %d, %Y'
    
    # Parse the date string using the input format
    date_obj = datetime.strptime(date_str, input_format)
    
    # Format the date object according to the desired output format
    formatted_date = date_obj.strftime(output_format)
    
    # Return the formatted date
    return formatted_date

In [47]:
df_fixtures_next5['Date'] = df_fixtures_next5['Date'].apply(convert_date)

C:\Users\Owner\AppData\Local\Temp\ipykernel_57356\2824450436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixtures_next5['Date'] = df_fixtures_next5['Date'].apply(convert_date)


In [48]:
df_fixtures_next5

,Date,League,Home team,Time,Away team
5,"Apr 17, 2024",UCL,Manchester City,01:00 PM,Real Madrid
6,"Apr 20, 2024",FAC,Manchester City,10:15 AM,Chelsea
7,"Apr 25, 2024",PRL,Brighton & Hove Albion,01:00 PM,Manchester City
8,"Apr 28, 2024",PRL,Nottingham Forest,09:30 AM,Manchester City
9,"May 04, 2024",PRL,Manchester City,10:30 AM,Wolverhampton Wanderers


In [ ]:
df['GAME_DATE'] + ': ' + df['VISITOR_TEAM_ABBREVIATION'] + ' @ ' + df['HOME_TEAM_ABBREVIATION'] + ' - ' + df['game_time_mountain']

In [52]:
df_fixtures_next5['display_msg'] = df_fixtures_next5['Date'] + ': ' + df_fixtures_next5['Away team'] + ' @ ' + df_fixtures_next5['Home team'] + ' - ' + df_fixtures_next5['Time']

C:\Users\Owner\AppData\Local\Temp\ipykernel_57356\1367554240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixtures_next5['display_msg'] = df_fixtures_next5['Date'] + ': ' + df_fixtures_next5['Away team'] + ' @ ' + df_fixtures_next5['Home team'] + ' - ' + df_fixtures_next5['Time']


In [64]:
df_fixtures_next5

,Date,League,Home team,Time,Away team,display_msg
5,"Apr 17, 2024",UCL,Manchester City,01:00 PM,Real Madrid,"Apr 17, 2024: Real Madrid @ Manchester City - ..."
6,"Apr 20, 2024",FAC,Manchester City,10:15 AM,Chelsea,"Apr 20, 2024: Chelsea @ Manchester City - 10:1..."
7,"Apr 25, 2024",PRL,Brighton & Hove Albion,01:00 PM,Manchester City,"Apr 25, 2024: Manchester City @ Brighton & Hov..."
8,"Apr 28, 2024",PRL,Nottingham Forest,09:30 AM,Manchester City,"Apr 28, 2024: Manchester City @ Nottingham For..."
9,"May 04, 2024",PRL,Manchester City,10:30 AM,Wolverhampton Wanderers,"May 04, 2024: Wolverhampton Wanderers @ Manche..."


In [65]:
df_fixtures_next5.display_msg.values[0]

'Apr 17, 2024: Real Madrid @ Manchester City - 01:00 PM'

In [68]:
df_fixtures_next5_fin = df_fixtures_next5[['display_msg']]
msg_list = df_fixtures_next5_fin['display_msg'].tolist()
final_msg = ', '.join(msg_list)
final_msg

'Apr 17, 2024: Real Madrid @ Manchester City - 01:00 PM, Apr 20, 2024: Chelsea @ Manchester City - 10:15 AM, Apr 25, 2024: Manchester City @ Brighton & Hove Albion - 01:00 PM, Apr 28, 2024: Manchester City @ Nottingham Forest - 09:30 AM, May 04, 2024: Wolverhampton Wanderers @ Manchester City - 10:30 AM'

In [62]:
df_fixtures_prev5['display_msg'] = df_fixtures_prev5['Date'] + ': ' + df_fixtures_prev5['Home team'] + ' ' + df_fixtures_prev5['Time'] + ' ' + df_fixtures_prev5['Away team']

C:\Users\Owner\AppData\Local\Temp\ipykernel_57356\1962091628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixtures_prev5['display_msg'] = df_fixtures_prev5['Date'] + ': ' + df_fixtures_prev5['Home team'] + ' ' + df_fixtures_prev5['Time'] + ' ' + df_fixtures_prev5['Away team']


In [63]:
df_fixtures_prev5

,Date,League,Home team,Time,Away team,display_msg
0,31/03/24,PRL,Manchester City,0 - 0,Arsenal,31/03/24: Manchester City 0 - 0 Arsenal
1,03/04/24,PRL,Manchester City,4 - 1,Aston Villa,03/04/24: Manchester City 4 - 1 Aston Villa
2,06/04/24,PRL,Crystal Palace,2 - 4,Manchester City,06/04/24: Crystal Palace 2 - 4 Manchester City
3,09/04/24,UCL,Real Madrid,3 - 3,Manchester City,09/04/24: Real Madrid 3 - 3 Manchester City
4,13/04/24,PRL,Manchester City,5 - 1,Luton Town,13/04/24: Manchester City 5 - 1 Luton Town
